<a href="https://colab.research.google.com/github/ratnesh003/HCLTech-Tasks/blob/main/Silver%20Badge%20Assignments/Assignment%201%20/HCLTech_ML_Assignment_2_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

Use any other open-source model in place of openai and do the summarization of the pdf document using Lang chain and the concept of RAG. use chromadb vector database for this

### Installing Dependencies

In [1]:
!pip install \
  requests \
  opentelemetry-api \
  opentelemetry-sdk \
  opentelemetry-proto \
  opentelemetry-exporter-otlp-proto-common \
  langchain \
  langchain-community \
  langchain-huggingface \
  chromadb \
  pypdf \
  sentence-transformers \
  transformers \
  accelerate \
  bitsandbytes

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other 

### Upload the PDF

In [2]:
from google.colab import files
from langchain.document_loaders import PyPDFLoader

# Prompt user to upload a PDF
uploaded = files.upload()

# Get the uploaded file name (assumes one PDF)
pdf_path = next(iter(uploaded))

# Load PDF
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"Loaded PDF: {pdf_path}")
print(f"Total pages loaded: {len(documents)}")

Saving azure_databricks_cookbook.pdf to azure_databricks_cookbook.pdf
Loaded PDF: azure_databricks_cookbook.pdf
Total pages loaded: 452


### create the chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,          # Large enough for semantic coherence
    chunk_overlap=300,        # Prevents abrupt cut-offs
    separators=[
        "\n\n",               # Paragraphs
        "\n",                 # Lines
        ".",                  # Sentences
        " ",                  # Words
        ""
    ]
)

split_docs = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(split_docs)}")


Total chunks created: 678


### Load the embedding model

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipython-input-3291538701.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Setup the chromaDB

In [5]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    persist_directory="/content/chroma_db"
)

vectorstore.persist()


ERROR:chromadb.telemetry.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
/tmp/ipython-input-1706748245.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


### Load the Mistral model

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

model.config.pad_token_id = tokenizer.pad_token_id

text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.3,
    repetition_penalty=1.1
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


### Setting up the text generation pipeline

In [7]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


/tmp/ipython-input-3763298684.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


### Setting up the retrieval configurations

In [8]:
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)


### Adding Query and prompt for the generation

In [ ]:
# Ask user for a custom query
query = input(
    "Enter your question about the uploaded PDF:\n"
).strip()

if not query:
    raise ValueError("Query cannot be empty.")

docs = retriever.get_relevant_documents(query)
context = "\n\n".join(doc.page_content for doc in docs)

prompt = f"""
You are a senior data engineer.

Using the context below, answer the question clearly and technically.
Do NOT repeat the context.
Do NOT include headings.

Context:
{context}

Question:
{query}

Answer:
"""

### Streamming the text generation

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread

MAX_INPUT_TOKENS = 8192

# Create streamer
streamer = TextIteratorStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True
)

# Tokenize input WITH attention mask
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=MAX_INPUT_TOKENS
).to(model.device)

# Background generation
thread = Thread(
    target=model.generate,
    kwargs={
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "streamer": streamer,
        "max_new_tokens": 512,
        "do_sample": True,          # REQUIRED for temperature
        "temperature": 0.3,
        "repetition_penalty": 1.1,
        "pad_token_id": tokenizer.pad_token_id
    }
)

thread.start()

# Stream answer tokens only
print("\nAnswer:\n")
for token in streamer:
    print(token, end="", flush=True)

### Listing the references used for answer generation

In [ ]:
print("\n\nReferences used from the document:\n")

for i, doc in enumerate(docs, 1):
    source = doc.metadata.get("source", "Unknown source")
    page = doc.metadata.get("page", "Unknown page")
    print(f"[{i}] Source: {source}, Page: {page}")